In [4]:
import re
import os.path as op

In [28]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [21]:
# load libraries and set plot parameters
import numpy as np
#import PrettyTable as pt

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 75

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

#plt.rcParams['text.usetex'] = True
#plt.rcParams['font.family'] = "serif"
#plt.rcParams['font.serif'] = "cm"
#plt.rcParams['text.latex.preamble'] = r"\usepackage{subdepth}, \usepackage{type1cm}"

import pandas as pd

import datetime as dt
import accounticon as acc

icon_acc_file = r'C:\Users\gonzalo\Projects\nmrlab\Inmracct.full'
nmr_usage_file = 'useNMR.csv'

In [10]:
# read accounting log file
with open(icon_acc_file) as au:
    contents = au.read()
    
exps = [x for x in re.split('-{44}\n', contents) if x.startswith('#Experiment')]    
    

In [11]:
def gen_records(exps, start=0, end=-1):
    if end == -1 :
        end = len(exps)
    records = []
    for exp in exps[start:end]:
        record =  {}
        
        items = [e.strip().split(":",1) for e in exp.split("\n") if e and not e.startswith("#") and ':' in e]
        #print(items)
        items = {k.strip():v.strip() for k,v in items}
        
        if not items.get('fileName'):
            continue

        if not items.get('timeOfStart') or not items.get('timeOfTermination'):
            continue

        it = re.findall(r'\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}', items['timeOfStart'])[0]
        record['Fecha Comienzo'] = dt.datetime.strptime(it, r'%m/%d/%Y %H:%M:%S')

        it = re.findall(r'\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}', items['timeOfTermination'])[0]
        record['Fecha Fin'] = dt.datetime.strptime(it, r'%m/%d/%Y %H:%M:%S')
                    
        record['Duración'] =  record['Fecha Fin'] - record['Fecha Comienzo']
        
        its = items['nameOfExperiment'].strip().split()
        record['Experimento'] = its[0] if len(its)==1 else its[1]
        
        record['Núcleo'] = items['NUCLEUS']
        record['Solvente'] = items['solvent']
        
        it = items['fileName'].split()
        record['Muestra'] = it[0]
        record['#Exp'] = it[1]
        record['Usuario'] = it[3].split('\\')[-1]
        
        records.append(record)
        
    return pd.DataFrame(records)

In [5]:
if op.exists('useNMR.csv'):
    with open('useNMR.csv') as ufile:
        df = pd.read_csv(ufile)
else:
    df = gen_records(exps)

In [12]:
df = gen_records(exps)
df.style

,#Exp,Duración,Experimento,Fecha Comienzo,Fecha Fin,Muestra,Núcleo,Solvente,Usuario
0,1,0 days 00:04:41,PROTON,2018-10-05 17:08:32,2018-10-05 17:13:13,guille.26763,1H,CDCl3,GValdomir
1,1,0 days 00:03:39,PROTON,2018-10-05 17:16:44,2018-10-05 17:20:23,ls.26761,1H,CDCl3,LauraS
2,1,0 days 00:03:39,PROTON,2018-10-05 17:20:23,2018-10-05 17:24:02,guille.26762,1H,CDCl3,GValdomir
3,1,0 days 00:03:47,PROTON,2018-10-05 17:24:03,2018-10-05 17:27:50,lp.26764,1H,CDCl3,GSerra
4,2,0 days 00:05:53,COSYGPSW,2018-10-05 17:27:50,2018-10-05 17:33:43,lp.26764,1H,CDCl3,GSerra
5,1,0 days 00:00:47,PROTON,2018-10-05 17:33:43,2018-10-05 17:34:30,mi.26765,1H,DMSO,MIncerti
6,1,0 days 00:05:09,PROTON,2018-10-05 17:34:31,2018-10-05 17:39:40,mi.26766,1H,CDCl3,MIncerti
7,1,0 days 00:03:39,PROTON,2018-10-05 17:50:04,2018-10-05 17:53:43,mi.26765,1H,DMSO,MIncerti
8,2,0 days 00:05:52,COSYGPSW,2018-10-05 17:53:43,2018-10-05 17:59:35,mi.26765,1H,DMSO,MIncerti
9,1,0 days 00:04:06,PROTON,2018-10-05 17:59:35,2018-10-05 18:03:41,mi.26766,1H,CDCl3,MIncerti


In [13]:
df.Usuario.count()

1775

In [14]:
df.Duración[0] / np.timedelta64(1, 'm')

4.683333333333334

In [15]:
inicio = df['Fecha Comienzo'].min()
final = df['Fecha Fin'].max()
tiempo_total = final - inicio

df['Día'] = df['Fecha Fin'].dt.day_name()

In [30]:
df.index

RangeIndex(start=0, stop=1775, step=1)

In [37]:
jan1 = pd.to_datetime('01/01/2019')
use2018 = df[df['Fecha Fin'] < jan1]

totime18 = jan1 - df['Fecha Comienzo'].min()
totuse18 = use2018.Duración.sum()
totuse18/totime18

0.574219835661079

In [18]:
u18_pv = pd.pivot_table(use2018, values='Duración', index='Usuario', aggfunc=['sum']).sort_values(('sum','Duración'), ascending=False)

#### Horas de uso por usuario en el período 5/10 - 31/12 

In [45]:
index = u18_pv.index.values
horas = u18_pv[('sum','Duración')]/np.timedelta64(1, 'h')
percent = u18_pv[('sum','Duración')]/totuse18*100

p = figure(y_range=index, 
           plot_width=800, 
           plot_height=600,
           x_range = (-1,240),
           x_axis_label='Uso Total (horas)',
           y_axis_label='Usuario')

p.hbar(y=index, 
       right=horas, 
       height=0.5, 
       left=0,
       color="firebrick")

# insertar porcentaje del total al final de cada barra o como un segundo eje x

p.text(horas+1, index, ['%.2f%%' % r for r in percent],
       text_font_size="8pt", text_align="left", text_baseline="middle")

show(p)

In [54]:
# top 3 users
top3 = horas.head(3).sum()/(totuse18/np.timedelta64(1, 'h'))*100


46.3555242938876

In [56]:
# day of week vs time vbar
u18i = use2018.set_index('Fecha Comienzo')
u18i.resample('D')

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, base=0]

In [ ]:
# day/night use vs day of week
# most popular experiment
# most popular experiment vs day of week
# total time vs experiment type

In [72]:
daily = u18i['Duración'].resample('D')
tot_daily = daily.sum()
tot_daily

Fecha Comienzo
2018-10-05   0 days 01:13:45
2018-10-06   0 days 00:00:00
2018-10-07   0 days 00:00:00
2018-10-08   0 days 16:54:15
2018-10-09   0 days 01:46:13
2018-10-10   0 days 12:33:41
2018-10-11   0 days 16:45:37
2018-10-12   0 days 12:02:46
2018-10-13   0 days 00:11:12
2018-10-14   0 days 00:00:00
2018-10-15   0 days 00:00:00
2018-10-16   0 days 10:57:56
2018-10-17   0 days 17:41:42
2018-10-18   1 days 04:25:35
2018-10-19   1 days 20:42:20
2018-10-20   0 days 00:00:00
2018-10-21   1 days 00:39:48
2018-10-22   1 days 00:22:32
2018-10-23   0 days 04:03:12
2018-10-24   0 days 22:22:45
2018-10-25   0 days 20:08:58
2018-10-26   1 days 00:10:30
2018-10-27   0 days 19:47:13
2018-10-28   0 days 18:47:18
2018-10-29   0 days 23:48:00
2018-10-30   0 days 22:20:39
2018-10-31   0 days 14:19:24
2018-11-01   1 days 00:00:10
2018-11-02   0 days 04:52:39
2018-11-03   0 days 00:00:00
                   ...      
2018-11-30   1 days 03:37:56
2018-12-01   0 days 00:14:01
2018-12-02   0 days 00:00:00

In [64]:
u18i['day_name'] = u18i['Fecha Fin'].dt.day_name()

In [69]:
daily_use = pd.pivot_table(u18i, values='Duración', index='day_name', aggfunc=['sum'])

days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6, 'Sunday':7}
key = daily_use.['DAY_OF_WEEK'].map(mapping)

horas = daily_use[('sum','Duración')]/np.timedelta64(1, 'h')
percent = daily_use[('sum','Duración')]/totuse18*100

p = figure(x_range=index, 
           plot_width=800, 
           plot_height=600,
           x_axis_label='Día de la Semana',
           y_axis_label='Horas de Uso')

p.vbar(x=index, 
       width=0.5, 
       top=horas, 
       bottom=0)

p.text(index, horas+1, ['%.2f%%' % r for r in percent],
       text_font_size="8pt", text_align="center", text_baseline="bottom")

show(p)

In [71]:
daily_use.index

Index(['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday'],
      dtype='object', name='day_name')